# German Credit Dataset

- 데이터를 나이브 베이즈 문제에 맞도록 간단하게 변환합니다.
- Binary 데이터들로 이루어진 대출 사기 데이터들로 부터 대출인지 아닌지 예측

In [115]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [116]:
# 데이터셋 불러오기
data_url = './fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [117]:
# ID열을 삭제
del df["ID"] 

# Label(Y_data)을 따로 저장
Y_data = df.pop("Fraud")

In [118]:
# as_matrix()함수를 통해 numpy array형태로 변환시켜 줍니다.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.as_matrix.html

Y_data = Y_data.as_matrix()
print("데이터확인", Y_data)
print("데이터타입", type(Y_data))

데이터확인 [ True False False  True False  True False False False  True False  True
  True False False False False False False False]
데이터타입 <class 'numpy.ndarray'>


C:\Users\john\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [119]:
df.head()
# 명목형 변수들이 있는데 인코딩이 아직 되어 있지 않다.

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


## One-Hot encoding

* 범주형 변수를 dummy변수로 변환

### 1. One-Hot Encoding

In [120]:
# 범주형 변수 처리 후 10개의 피처를 얻을 수 있음.

x_df = pd.get_dummies(df)
x_df.head()

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


In [121]:
x_data = x_df.as_matrix()
x_data

C:\Users\john\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

#### Q1. as_matrix()함수를 통해 하고자 하는 것?
데이터프레임보다 ndarray에서 행별로 접근하는 것이 더 용이하다. 편리한 연산을 위해 자료 구조를 변경해준다.

In [122]:
Y_data == True # boolean index

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [123]:
len(set(Y_data)) #True OR False

2

## Naive bayes classifier

* P(Y)
* P(X1, X2, ..., Xn)
* P(Y|X1, X2, X3, ..., Xn)
* P(X1|Y), P(X2|Y), ... P(Xn|Y)

### P(Y1), P(Y0) 구하기
P_Y_True는 P(Y1), P_Y_False는 P(Y2)

In [124]:
# P(Y1), P(Y0)
# P(Y1) = count(Y1) / count(Y)

P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True, P_Y_False

(0.3, 0.7)

1. 이 튜토리얼에서는 **index를 이용합니다.**
2. y가 1일 경우, y가 0일 경우를 구해줘야 합니다.
3. np.where은 list.index()와 같은 기능을 합니다.

In [125]:
ix_Y_True = np.where(Y_data) # Y_data == True인 인덱스를 뽑아줍니다.
ix_Y_False = np.where(Y_data==False)

ix_Y_True, ix_Y_False
# np.where을 사용해서 Y가1일 때와 0일 때 각각의 인덱스 값을 얻을 수 있게 되었습니다.

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

In [126]:
# 조건부확률
# P(X|Y) = count(X_cap_Y) / count(Y) 
# cap = 교집합

### productP(X|Yc) 구하기

* product * P(X|Y1)
* product * P(X|Y2)

In [127]:
# p_x_y_true는 y가 true일 때 xi 들의 확률 P(xi|y1)
p_x_y_true = x_data[ix_Y_True].sum(axis=0) / sum(Y_data == True)
p_x_y_true

# Q. 앞에 식이 P(X_cap_Y1)인 것 같은데 왜 뒤에 sum(Y_data == True) 필요한가? 
# A. Y1일 때의 조건부 확률을 만들기 위해 count(Y1)을 구해준 것

array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
       0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333])

In [128]:
# P(xi/y2)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0) / sum(Y_data==False))
p_x_y_false

array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
       0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714])

In [129]:
x_data_true = x_data[ix_Y_True].sum(axis=0)
x_data_false = x_data[ix_Y_False].sum(axis=0)

In [130]:
x_data_true

array([1, 3, 1, 1, 0, 1, 5, 0, 4, 2], dtype=uint32)

In [131]:
# 총 10개의 값에 대해서 확률을 구해준다.
xi_t = x_data.sum(axis=0)/len(x_data)
xi_t

array([0.35, 0.35, 0.05, 0.25, 0.1 , 0.05, 0.85, 0.05, 0.75, 0.2 ])

In [132]:
import math
x_test = [0,1,0,0,0,1,0, 0,1,0]

# 분모는 같으므로 비교할 필요가 없어서 생략
# 확률값을 곱하면 점점 너무 작아져서 로그를 사용함
log_p_y_true_test = math.log10(P_Y_True + p_x_y_true.dot(x_test)) 
log_p_y_false_test = math.log10(P_Y_False + p_x_y_false.dot(x_test))

log_p_y_true_test, log_p_y_false_test

(0.21307482530885122, 0.24832364514797822)

In [133]:
log_p_y_true_test < log_p_y_false_test

True

## 2. Smoothing을 통해 P(Y=1|X)의 확률과 P(Y=0|X)의 확률 값을 비교

In [134]:
# 값이 0에 수렴하는 것을 방지하기 위해 smoothing_p 파라미터를 사용
import math
x_test = [0,1,0,0,0,1,0, 0,1,0]

smoothing_p = 2 #hyperparameter. if laplace, smoothing_p = 1

sm_p_y_true_test = (P_Y_True * (x_data_true.dot(x_test)+smoothing_p)/(len(x_data_true)+smoothing_p*x_data.shape[1]))
sm_p_y_false_test = (P_Y_False * (x_data_false.dot(x_test)+smoothing_p)/(len(x_data_false)+smoothing_p*x_data.shape[1]))

sm_p_y_true_test, sm_p_y_false_test

(0.1, 0.3966666666666666)

In [135]:
p_y_true_test < p_y_false_test

True